In [2]:
#!pip install QuantumRingsLib==0.9.11 matplotlib==3.10.0

In [3]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math

provider = QuantumRingsProvider(
    token='rings-128.TdVKnV6k5pEiSIyMIRYG05gLXKmUOJkq',
    name='g.scorpiosky@gmail.com'
)
backend = provider.get_backend("scarlet_quantum_rings")

shots = 1024

provider.active_account()

{'name': 'g.scorpiosky@gmail.com',
 'token': 'rings-128.TdVKnV6k5pEiSIyMIRYG05gLXKmUOJkq',
 'max_qubits': '128'}

In [4]:
!pip install qiskit[visualization]

  Using cached qiskit-1.3.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached scipy-1.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.4.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pydot-3.0.4-py3-none-any.whl.metadata (10 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
Using ca

In [5]:
!pip install quantumrings-toolkit-qiskit

  Using cached quantumrings_toolkit_qiskit-0.1.9-py3-none-any.whl.metadata (15 kB)
Using cached quantumrings_toolkit_qiskit-0.1.9-py3-none-any.whl (52 kB)


In [6]:
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 72.9 MB/s eta 0:00:00:00:01


In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT

from qiskit_aer.primitives import Sampler
from qiskit.visualization import plot_histogram

In [8]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for _ in range(j):
        a = np.mod(a**2, N)
    return a

def qft_dagger(n):
    """Creates an n-qubit inverse Quantum Fourier Transform circuit."""
    qc = QFT(num_qubits=n, do_swaps=False).inverse()
    return qc.to_gate(label="QFT†")

In [9]:
from fractions import Fraction
from math import gcd,ceil,log2, pi

def c_amod15(a,N):
    """
    Controlled multiplication by a mod N using QFT instead of SWAP.
    """
    Q = ceil(log2(N))
    U = QuantumCircuit(Q)
    for i in range(Q // 2):
        U.swap(i, Q-i-1)

    if a//2 ==1:
        for q in range(Q):      
            U.x(q)
    
    U.append(qft_dagger(Q), range(Q))
    print(f"num_qubits:{U.num_qubits}")

    #display(U.draw("mpl"))
    U = U.to_gate()
    U.name = f"{a} mod {Q}"
    c_U = U.control()
    return c_U

In [12]:
def phase_estimation(
        controlled_operation: QuantumCircuit,
        psi_prep: QuantumCircuit,
        precision: int
    ):
    """
    Carry out phase estimation on a simulator.
    Args:
        controlled_operation: The operation to perform phase estimation on,
                              controlled by one qubit.
        psi_prep: Circuit to prepare |ψ>
        precision: Number of counting qubits to use
    Returns:
        float: Best guess for phase of U|ψ>
    """
    control_register = QuantumRegister(precision)
    output_register = ClassicalRegister(precision)

    target_register = QuantumRegister(psi_prep.num_qubits)
    qc = QuantumCircuit(control_register, target_register, output_register)

    # Prepare |ψ>
    qc.compose(psi_prep,
               qubits=target_register,
               inplace=True)

    # Do phase estimation
    theta = 0.7
    for index, qubit in enumerate(control_register):
        qc.h(qubit)
        qc.cp(2 * pi * theta*(index+1), qubit, target_register)
    qc.barrier()

    qc.compose(
        QFT(precision, inverse=True),
        qubits=control_register,
        inplace=True
    )

    # Count the operations
    gate_counts = qc.size()
    print(f"gate_counts:{gate_counts}")

    qc.measure(control_register, output_register)

    measurement = Sampler().run(qc, shots=1).result().quasi_dists[0].popitem()[0]
    return measurement / 2**precision

In [11]:
import semiprimes
data = semiprimes.semiprimes
items = list(data.items())
a = items[0][0]
N = items[0][1]

Q = ceil(log2(N))
psi_prep = QuantumCircuit(Q)
psi_prep.x(0)
#display(psi_prep.draw("mpl"))

a,N,Q

(8, 143, 8)

In [13]:
from fractions import Fraction
from math import gcd,ceil,log2,pi

#a = 8
#N = 15

import time
start_time = time.time()

FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nAttempt {ATTEMPT}")

    phase = phase_estimation(
        c_amod15(a,N),
        psi_prep,
        precision=Q
    )
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    if phase != 0:
        # Guess for a factor is gcd(x^{r/2} - 1 , 15)
        guess = gcd(a ** (r // 2) - 1, N)
        if guess not in [1, N] and (N % guess) == 0:
            # Guess is a factor!
            print(f"Non-trivial factor found: {guess}")
            FACTOR_FOUND = True


end_time = time.time()  # End time
elapsed_time = end_time - start_time  # Calculate elapsed time

print(f"Elapsed time: {elapsed_time:.2f} seconds")


Attempt 1
num_qubits:8
gate_counts:74
Non-trivial factor found: 11
Elapsed time: 0.18 seconds


/tmp/ipykernel_218/3185486489.py:16: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation(
